# BIO 310 Homework 1 - Semi Global Sequence Alignment

In this homework, you will implement Semi Global Sequence Alignment. You are given a partial Global Alignment code. You are expected to modify some parts or fill in the blanks to implement it.

There are 2 main excercise that you will encounter in this Homework. For the detailed explanation, please refer to the Homework PDF.

* **MODIFY STARTS:** When you see MODIFY, it means this part is for Global Alignment. Therefore, you should convert it to Semi-Global Alignment

* **TODO STARTS:** You should implement this part or fill in the blank (...) parts for Semi-Global Alignments.

In [1]:
# In Google Colab, Numpy and Pandas are installed already. In case you want to use them on your local computer, you can install them with pip

# !pip install numpy
# !pip install pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [4]:
 ###################### TODO START ######################

def initialize_penalties(match_score, mismatch_score, seq1_start_gap, seq2_start_gap,
                         seq1_end_gap, seq2_end_gap, internal_gap):
    match = match_score
    mismatch = mismatch_score


    gap_penalties = {
        'seq1_start': seq1_start_gap,
        'seq2_start': seq2_start_gap,
        'seq1_end': seq1_end_gap,
        'seq2_end': seq2_end_gap,
        'gap_internal': internal_gap
    }

    # The direction dictionary will be used for backtracking (Do not change)
    direction_dict = {
        'diagonal': 0,
        'left': 1,
        'up': 2
    }

    return match, mismatch, gap_penalties, direction_dict


 ###################### TODO END ######################

In [5]:
def read_sequences(filename):
    """
    Read the file and return two sequences in list format

    Args:
        filename (string): name of the sequence file
    """
    with open(filename) as fn:
        seq1 = fn.readline().strip()
        seq2 = fn.readline().strip()
        match = int(fn.readline().split(':')[1].strip())
        mismatch = int(fn.readline().split(':')[1].strip())
        internal_gap = int(fn.readline().split(':')[1].strip())
        seq1_start_gap = int(fn.readline().split(':')[1].strip())
        seq2_start_gap = int(fn.readline().split(':')[1].strip())
        seq1_end_gap = int(fn.readline().split(':')[1].strip())
        seq2_end_gap = int(fn.readline().split(':')[1].strip())
    seq1 = list(seq1)
    seq2 = list(seq2)


    return seq1, seq2, match, mismatch, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap

In [6]:
###################### TODO START ######################

def isMatch(a, b, match_score, mismatch_score):
    """
    Checks if two characters are the same and returns a score

    Args:
        a (string): A nucleotide from one sequence
        b (string): A nucleotide from another sequence
        match_score (int): Match Score
        mismatch_score (int): Mismatch Score

    Return:
        Score (int): If they match, return match score; otherwise, return mismatch score.
    """
    return match_score if a == b else mismatch_score

###################### TODO END ######################


In [7]:
def score_matrix(seq1, seq2, match, mismatch, gap_penalties, direction_dict):
    """
    Creates score and direction matrices for alignment of two sequences

    Args:
        seq1 (list): DNA Sequence 1
        seq2 (list): DNA Sequence 2
        match_score (int): Match Score
        mismatch_score (int): Mismatch Score
        gap_penalties (dict): Contains different gap penalties

    Return:
        df_scores (pd.DataFrame): Score Matrix
        df_directions (pd.DataFrame): Direction Matrix
        max_loc (Tuple): Coordinates of the max position
    """

    len_seq1 = len(seq1)
    len_seq2 = len(seq2)

    # Zero-filled matrices for scores and directions
    scores = np.zeros((len_seq1 + 1, len_seq2 + 1))
    directions = np.zeros((len_seq1 + 1, len_seq2 + 1))


    ###################### MODIFY START ######################

    # Fill first row of scores and directions for seq 1 gap:
    for i in range(1, len(seq2) + 1):
        scores[0][i] = scores[0][i-1] + gap_penalties['seq2_start']
        directions[0][i] = direction_dict['left']

    # Fill first column of scores and directions for seq 2 gap:
    for j in range(1, len(seq1) + 1):
        scores[j][0] = scores[j-1][0] + gap_penalties['seq1_start']
        directions[j][0] = direction_dict['up']

    ###################### MODIFY END ######################


    # Fill the internal cells of scores and directions:
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match_score = scores[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            left_score = scores[i][j-1] + gap_penalties['gap_internal']
            up_score = scores[i-1][j] + gap_penalties['gap_internal']

            move_score = max(match_score, left_score, up_score)

            ###################### TODO END ######################

            if move_score == match_score:
                scores[i][j] = match_score
                directions[i][j] = direction_dict['diagonal']
            elif move_score == left_score:
                scores[i][j] = left_score
                directions[i][j] = direction_dict['left']
            else:
                scores[i][j] = up_score
                directions[i][j] = direction_dict['up']


    ###################### MODIFY START ######################

    # max_loc is the starting location of the backtracking.
    last_row_max = max(scores[-1])
    last_col_max = max(scores[:, -1])
    if last_row_max > last_col_max:
        max_loc = (len(seq1), np.argmax(scores[-1]))
    else:
        max_loc = (np.argmax(scores[:, -1]), len(seq2))

    ###################### MODIFY END ######################


    # Our matrix size is (m+1, n+1). But our sequence lengths are m and n.
    # Add * at the beginnings of sequences
    if len(seq1) !=len(scores):
        seq1.insert(0,'*')
        seq2.insert(0,'*')
    # Convert them numpy array
    seq1_np = np.array(seq1)
    seq2_np = np.array(seq2)

    # To see better, convert them as pandas dataframe
    scores_matrix=np.asmatrix(scores)
    df_scores = pd.DataFrame(scores_matrix, columns= seq2_np, index = seq1_np)

    directions_matrix = np.asmatrix(directions)
    df_directions = pd.DataFrame(directions_matrix, columns= seq2_np, index = seq1_np)


    return df_scores, df_directions, max_loc


In [8]:
def backtrack(scores, directions, max_loc, seq1, seq2, gap_penalties):
    # Coordinates to track. Start from max_loc until the first row or column
    i, j = max_loc

    # Final information to print the alignment
    sequence_1 = ''
    sequence_2 = ''
    alignment_info = ''

    ###################### TODO START ######################
    # Start from [i, j] and construct the alignment
    while i > 0 or j > 0:
        if directions.iloc[i, j] == 0:  # Diagonal
            sequence_1 = seq1[i - 1] + sequence_1
            sequence_2 = seq2[j - 1] + sequence_2
            if seq1[i - 1] == seq2[j - 1]:  # Match
                alignment_info = '|' + alignment_info
            else:
                alignment_info = '.' + alignment_info
            i -= 1
            j -= 1
        elif directions.iloc[i, j] == 1:  # Left
            sequence_1 = '-' + sequence_1
            sequence_2 = seq2[j - 1] + sequence_2
            alignment_info = ' ' + alignment_info
            j -= 1
        else:  # Up
            sequence_1 = seq1[i - 1] + sequence_1
            sequence_2 = '-' + sequence_2
            alignment_info = ' ' + alignment_info
            i -= 1

    # Add the start gaps to the alignment (if any)
    while i > 0:
        sequence_1 = seq1[i - 1] + sequence_1
        sequence_2 = '-' + sequence_2
        alignment_info = ' ' + alignment_info
        i -= 1

    while j > 0:
        sequence_1 = '-' + sequence_1
        sequence_2 = seq2[j - 1] + sequence_2
        alignment_info = ' ' + alignment_info
        j -= 1
    ###################### TODO END ######################

    # Reverse the sequences
    sequence_1 = sequence_1[::-1]
    sequence_2 = sequence_2[::-1]
    alignment_info = alignment_info[::-1]

    return sequence_1, sequence_2, alignment_info


In [9]:
###################### TODO START ######################

def semi_global_alignment(filename):

    # 1. Read File
    seq1, seq2, match_score, mismatch_score, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap = read_sequences(filename)

    # 2. Initialize Penalties
    match, mismatch, gap_penalties, direction_dict = initialize_penalties(match_score, mismatch_score, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap)

    # 3. Get score and direction matrix
    df_scores, df_directions, max_loc = score_matrix(seq1, seq2, match, mismatch, gap_penalties, direction_dict)

    # 4. Backtrack
    sequence_1, sequence_2, alignment_info = backtrack(df_scores, df_directions, max_loc, seq1, seq2, gap_penalties)
    sequence_1 = sequence_1.replace('*', '')
    sequence_2 = sequence_2.replace('*', '')
    alignment_info = alignment_info.replace('*', '')
    # 5. Print the results (Don't Change This)
    print(f'Match Score: {match_score}')
    print(f'Mismatch Score: {mismatch_score}')
    print(f'Internal Gap penalty: {internal_gap}')
    print(f'Gaps penalty at the start of string 1: {seq1_start_gap}')
    print(f'Gaps penalty at the start of string 2: {seq2_start_gap}')
    print(f'Gaps penalty at the end of string 1: {seq1_end_gap}')
    print(f'Gaps penalty at the end of string 2: {seq2_end_gap}')
    print()
    print(f'{sequence_1}\n{alignment_info}\n{sequence_2}')

###################### TODO END ######################

# Run the tests in the cells below

## Test 1

In [11]:
semi_global_alignment('test1.fasta')

Match Score: 15
Mismatch Score: -7
Internal Gap penalty: -7
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

GTACC-GTTA
|||.. |.   
GTATACG---


## Test 2

In [12]:
semi_global_alignment('test2.fasta')

Match Score: 10
Mismatch Score: -2
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -8
Gaps penalty at the start of string 2: -8
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

GTACC-GTTA
|||.. |   |
GTATACG---


## Test 3

In [13]:
semi_global_alignment('test3.fasta')

Match Score: 10
Mismatch Score: -3
Internal Gap penalty: -2
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -10
Gaps penalty at the end of string 2: -10

GTATCC-CGAAAGTGTCGTCGGAACA-GGAATAGT---CT-CTA---
|. . . ||.||.||. .     ||. |. |.||.   |. ||.    
GA-C-GCCGGAACTGC-A-----ACCAGA-ACAGCCGTCCTCTG-TC


## Test 4

In [14]:
semi_global_alignment('test4.fasta')

Match Score: 10
Mismatch Score: -2
Internal Gap penalty: -7
Gaps penalty at the start of string 1: -7
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -7
Gaps penalty at the end of string 2: 0

GTATCCCGAAAGTGTCGTCGGAACAGGAATAGTCTCTA
|. ..|||.||.||....|.||||||...|..|||.|.|
GA-CGCCGGAACTGCAACCAGAACAGCCGTCCTCTGTC


## Test 5

In [15]:
semi_global_alignment('test5.fasta')

Match Score: 5
Mismatch Score: -2
Internal Gap penalty: -2
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

AGTTTAGC
||||.    
AGTT----


## Test 6

In [16]:
semi_global_alignment('test6.fasta')

Match Score: 1
Mismatch Score: -1
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

GCTGAACTGAACC
|||.||||...   
GCTAAACTTG---


## Test 7

In [17]:
semi_global_alignment('test7.fasta')

Match Score: 1
Mismatch Score: -1
Internal Gap penalty: -1
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

CITAMROFNIOIB
||||||||||.   
CITAMROFNI---


## Test 8

In [18]:
semi_global_alignment('test8.fasta')

Match Score: 8
Mismatch Score: -2
Internal Gap penalty: -4
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -1
Gaps penalty at the end of string 2: 0

GG-ACACTTAC
|. ||.||.  |
GCGACGCTC--


## Test 9

In [19]:
semi_global_alignment('test9.fasta')

Match Score: 10
Mismatch Score: -1
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -2
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

NAMUHE---RAEW
|||||.   |||||
NAMUHTONERAEW
